# Exercício: predição de prazo de entrega

**Objetivo:** predizer prazo de entrega usando a base de dados do arquivo `processed_olist_orders.csv`

In [ ]:
! pip install geopy matplotlib pandas scikit-learn seaborn

In [ ]:
from geopy.distance import distance
import numpy as np
import pandas as pd

## bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Leitura dos dados

In [ ]:
orders_df = pd.read_csv("processed_olist_orders.csv")

* transforma coluna de datas relevantes em timestamp

In [ ]:
orders_df["order_purchase_timestamp"] = pd.to_datetime(orders_df["order_purchase_timestamp"])
orders_df["order_delivered_carrier_date_timestamp"] = pd.to_datetime(pd.to_datetime(orders_df["order_delivered_carrier_date"]).dt.strftime("%Y-%m-%d"))
orders_df["order_delivered_customer_date_timestamp"] = pd.to_datetime(pd.to_datetime(orders_df["order_delivered_customer_date"]).dt.strftime("%Y-%m-%d"))

In [ ]:
orders_df.head()

## Inspeção de algumas colunas

#### `order_status`: status do pedido

In [ ]:
orders_df["order_status"].value_counts().reset_index().rename(columns={"index": "order_status", "order_status": "count"})

### Dados por estado do `customer` e `seller`

In [ ]:
states_count = orders_df["customer_state"].value_counts().reset_index().rename(columns={"index": "state", "customer_state": "customer"}) \
    .merge(orders_df["seller_state"].value_counts().reset_index().rename(columns={"index": "state", "seller_state": "seller"}),
           on="state", how="left").fillna(0)
states_count_melted = pd.melt(states_count, id_vars="state", var_name="user_type", value_name="count")

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=120)
ax = fig.add_subplot(111)
sns.barplot(x="state", y="count", hue="user_type", data=states_count_melted, palette="rainbow", ax=ax)
sns.despine()

### Distribuição de `freight_value`: valor do frete

In [ ]:
orders_df[["freight_value"]].describe()

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=120)
ax = fig.add_subplot(111)
sns.boxplot(x="freight_value", data=orders_df, palette="rainbow", ax=ax)
sns.despine()

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=120)
ax = fig.add_subplot(111)
sns.histplot(x="freight_value", data=orders_df, palette="rainbow", ax=ax)
sns.despine()

#### Tarefa: que tal explorar mais colunas?

Por exemplo:

* qual é o intervalo de datas para o qual temos dados?
    * mínimo e máximo de da coluna `order_purchase_timestamp` (**Dica**: use o `.describe()` como foi usado para a coluna `freight_value` anteriormente)


* quais são as cidades (`customer_city` e `seller_city`) com mais pedidos? (**Dica**: use o `.value_counts()` como foi usado para a coluna `order_status` anteriormente) 

<!-- 
display(orders_df[["order_purchase_timestamp"]].describe())

display(orders_df["customer_city"].value_counts().reset_index().rename(columns={"index": "customer_city", "customer_city": "count"}).head(n=10))

display(orders_df["seller_city"].value_counts().reset_index().rename(columns={"index": "seller_city", "seller_city": "count"}).head(n=10)) 
-->

In [ ]:
####

### Apenas dados com status `delivered` devem ser mantidos no dataset

**Tarefa**: mantenha no dataframe `orders_df` somente os dados cujo valor na coluna `order_status` é igual a `"delivered"`.

<!-- 
orders_df = orders_df[orders_df["order_status"] == "delivered"]
-->

In [ ]:
orders_df = ####

## Criação de features

Exemplos:
* `same_city`: se `customer_city` é igual a `seller_city`
* `same_state`: se `customer_state` é igual a `seller_state`
* `zip_code_prefix_match`: número de prefixos iguais entre `customer_zip_code_prefix` e `seller_zip_code_prefix` (ex. `012345` e `01355` têm `zip_code_prefix_match` = 2)
* `geo_distance`: distância calculada a partir da latitude e longitude entre `customer` e `seller`
* `from_to`: combinação de `seller_state` e `customer_state` (ex. se `seller_state` é `SP` e `customer_state` é `RJ`, então, `from_to` = `SP->RJ`)

**Tarefa**: crie as colunas `same_city` e `same_state`.

**Dica 1**: `==` pode ser usado para comparar se duas colunas são iguais. Por exemplo: `df["a"] == df["b"]` devolve `True` nas linhas em que a coluna `a` é igual à coluna `b` e `False` nas linhas em que os valores são diferentes.

**Dica 2**: para transformar uma coluna cujos valores são `True` e `False` em uma coluna de `1` e `0`, basta fazer uma conversão usando `.astype(int)`. Por exemplo: `df["coluna_booleana"].astype(int)` faz com que todos os valores `True` sejam transformados em `1` e todos os valores `False`, em `0`.

<!-- 
orders_df["same_city"] = (orders_df["customer_city"] == orders_df["seller_city"]).astype(int)
orders_df["same_state"] = (orders_df["customer_state"] == orders_df["seller_state"]).astype(int)
 -->

In [ ]:
orders_df["same_city"] = ###
orders_df["same_state"] = ###

In [ ]:
orders_df["from_to"] = orders_df["customer_state"] + "->" + orders_df["seller_state"]

In [ ]:
for colname in ["customer_zip_code_prefix", "seller_zip_code_prefix"]:
    orders_df[f"{colname}_list"] = orders_df[colname].astype(str).str.zfill(5).apply(list)

def calc_prefix_match(row):
    customer_prefix_list = row["customer_zip_code_prefix_list"]
    seller_prefix_list = row["seller_zip_code_prefix_list"]
    num_matches = 0
    for i in range(5):
        if customer_prefix_list[i] != seller_prefix_list[i]:
            break
        num_matches += 1
    return num_matches
    
orders_df["zip_code_prefix_match"] = orders_df.apply(calc_prefix_match, 1)

In [ ]:
def calc_distance(row):
    customer_lat_long = (row["customer_lat"], row["customer_long"])
    seller_lat_long = (row["seller_lat"], row["seller_long"])
    return distance(customer_lat_long, seller_lat_long).km

In [ ]:
orders_df["geo_distance"] = orders_df.apply(calc_distance, 1)

#### Tarefa: que outras features podemos criar?

Por exemplo:

* `carrier_date_day_of_week`: dia da semana em que o pedido foi postado (`order_delivered_carrier_date`). **Dica**: Você pode usar o [.isoweekday()](https://docs.python.org/pt-br/3/library/datetime.html#datetime.date.isoweekday) da biblioteca `datetime`, que retorna o dia da semana, sendo que Segunda é 1 e Domingo é 7.

* `carrier_date_is_weekday`: `1` se o pedido foi postado (`order_delivered_carrier_date`) em um dia útil (segunda a sexta) e `0` se foi no fim de semana. Neste caso, uma sofisticação seria incluir dados sobre o calendário para saber quando foi feriado.

* `customer_region`: região (norte, nordeste, centro-oeste, sudeste, sul) a que pertence o estado em que o `customer` está (`customer_state`). Você pode consultar as regiões na [Wikipedia](https://pt.wikipedia.org/wiki/Regiões_do_Brasil).

* `seller_region`: região (norte, nordeste, centro-oeste, sudeste, sul) a que pertence o estado em que o `seller` está (`seller_state`)

<!-- 
display(orders_df[["order_purchase_timestamp"]].describe())

display(orders_df["customer_city"].value_counts().reset_index().rename(columns={"index": "customer_city", "customer_city": "count"}).head(n=10))

display(orders_df["seller_city"].value_counts().reset_index().rename(columns={"index": "seller_city", "seller_city": "count"}).head(n=10)) 
-->

In [ ]:
###

## Formato da coluna target

A coluna `target` é o prazo de entrega. Para calculá-lo, vamos considerar a quantidade de dias entre a postagem (`order_delivered_carrier_date`) e o recebimento (`order_delivered_customer_date`) pelo `customer`.

In [ ]:
def calc_diff_dates(row):
    carrier_date = row["order_delivered_carrier_date_timestamp"]
    customer_date = row["order_delivered_customer_date_timestamp"]
    return (customer_date - carrier_date).days

In [ ]:
orders_df["target"] = orders_df.apply(calc_diff_dates, 1)

#### Como é a distribuição do valor do target?

* olhe os valores mínimo e máximo do target

In [ ]:
orders_df[["target"]].describe()

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=120)
ax = fig.add_subplot(111)
sns.boxplot(x="target", data=orders_df, palette="rainbow", ax=ax)
p99 = orders_df["target"].quantile(0.99)
plt.axvline(x=p99, linestyle='--', color='cornflowerblue')
plt.text(p99 + 2, -0.4, f'99 percentil = {p99}', color='cornflowerblue')
sns.despine()

### O que seria um target estranho?

* valor negativo: entregou antes de postar

* zero: embora não seja impossível, não parece plausível

In [ ]:
len(orders_df[orders_df["target"] <= 0])

* vamos remover esses valores do dataset

In [ ]:
orders_df = orders_df[orders_df["target"] > 0]

## Preparação dos dados

In [ ]:
full_dataset = orders_df.reset_index(drop=True)

### Divisão do dataset em treino e teste

**Tarefa:** Faça a divisão no tempo, deixando 20% para nossa base de teste. **Dica:** use [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) da biblioteca sklearn com o parâmetro `shuffle = False`.


<!-- 
train_test_split(full_dataset, test_size=.2, random_state=0, shuffle=False)
 -->

In [ ]:
datasets = {}

In [ ]:
datasets["train"], datasets["test"] = ####

### Processamento de features

Features categóricas: média do prazo de entrega (`target`) no dataset de treino
* `from_to`
* `customer_city`

Features binárias: sem tratamento
* `same_city`
* `same_state`

Features numéricas: normalização de valores usando o [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
* `zip_code_prefix_match`
* `customer_lat`
* `customer_long`
* `seller_lat`
* `seller_long`
* `geo_distance`

**Atenção:** caso tenha criado outras features, lembre-se de incluí-las no local correto, de acordo com o tipo de seus valores.

**Tarefa**: Preencha o dicionário abaixo, colocando as colunas apropriadas nas listas correspondentes a cada uma das chaves.

<!-- 
features = {
    "categorical": ["from_to", "customer_city"],
    "binary": ["same_city", "same_state"],
    "numerical": ["zip_code_prefix_match", "customer_lat",
                 "customer_long", "seller_lat", "seller_long", "geo_distance"]
}
 -->

In [ ]:
features = {
    "categorical": [###],
    "binary": [###],
    "numerical": [###]}

In [ ]:
def process_numerical_feature(df, colname, scaler):
    df[f"{colname}_scaled"] = scaler.transform(df[[colname]]).reshape(-1)
    return df.drop(colname, 1)

def scale_numerical_cols(df, cols, scalers):
    for colname in cols:
        df = process_numerical_feature(df, colname, scalers[colname])
    return df

In [ ]:
def create_avg_categorical_cols(df, cols, all_categories):
    for colname in cols:
        from_to_values, mean_val = all_categories[colname]
        df[f"{colname}_avg"] = df[colname].apply(lambda val: from_to_values.get(val, mean_val))
    return df

In [ ]:
for dataset_type in ["train", "test"]:
    df = datasets[dataset_type]
    if dataset_type == "train":
        all_categories = {
            col: (df.groupby(col).agg({"target": "mean"})["target"].to_dict(), df["target"].mean()) for col in features["categorical"]
        }
        scalers = {
            col: MinMaxScaler().fit(df[[col]]) for col in features["numerical"]
        }
    df = create_avg_categorical_cols(df, features["categorical"], all_categories)
    datasets[dataset_type] = scale_numerical_cols(df, features["numerical"], scalers)

#### Informações do dataset de treino

In [ ]:
feature_cols = features["binary"] + [f"{col}_avg" for col in features["categorical"]] + [f"{col}_scaled" for col in features["numerical"]]

In [ ]:
datasets["train"][feature_cols].describe()

In [ ]:
datasets["train"][feature_cols].info()

## Construção do modelo

Usamos aqui como exemplo um modelo linear, mas fique à vontade para tentar com outro modelo, como por exemplo, uma [árvore de decisão](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html).

In [ ]:
model = LinearRegression()

**Tarefa:** Faça o `.fit` do modelo, passando como parâmetros:
* o dataset de treino (`datasets["train"]`) com as colunas `features_cols`
* o dataset de treino (`datasets["train"]`) com a coluna `target`.

<!-- 
model = model.fit(datasets["train"][feature_cols], datasets["train"]["target"])
 -->

In [ ]:
model = ###

### Métricas para avaliação

Para regressão, é comum utilizarmos uma das seguintes métricas para avaliar nosso modelo:

* `MAE` ([Mean Absolute Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error)): é a média do valor absoluto da diferença entre o valor predito e o valor real.

$$MAE = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y_i}| $$


* `RMSE` (Root Mean Squared Error): é a raiz quadrada da média do quadrado da diferença entre o valor predito e o valor real - por elevar ao quadrado o erro, penaliza erros maiores.

$$RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(y_i - \hat{y_i}\right)^2} $$

**Nota:** a biblioteca `scikit-learn` tem implementado o `MSE` ([Mean Squared Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error)). A cálculo do `MSE` segue: $MSE = \frac{1}{n} \sum_{i=1}^n \left(y_i - \hat{y_i}\right)^2$. Para calcular o `RMSE`, basta tirar a raiz quadrada desse valor.

* `r2` ([R2 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score)): é uma medida que indica o quanto o modelo é melhor em relação a um modelo que faz a predição com base no valor médio do target. Seu valor é entre menos infinito ($-\infty$) e 1.

$$R^2 = 1 - \frac{MSE}{Var(y)} = 1 - \frac{\frac{1}{n} \sum_{i=1}^n \left(y_i - \hat{y_i}\right)^2}{\frac{1}{n} \sum_{i=1}^n \left(y_i - \bar{y}\right)^2}$$

As fórmulas acima consideram:

* $n$: o número de exemplos avaliados;
* $y_i$: o valor real de `target` para o exemplo $i$;
* $\hat{y_i}$: o valor predito para o `target` para o exemplo $i$;
* $\bar{y}$: o valor médio dos valores reais de `target`.

**Tarefa**: Complete a implementação da função `rmse`.

**Dica 1**: use a função `mean_squared_error` do `scikit-learn` que já está importada.

**Dica 2**: a função da biblioteca `numpy` (aqui importada sob o apelido de `np`), [np.sqrt](https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html) retorna a raiz quadrada do valor que lhe é passado como parâmetro.

<!-- 
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
     -->

In [ ]:
def rmse(y_true, y_pred):
    ### 
    return ###

**Tarefa**: Faça a predição do dataset de teste (`datasets["test"][feature_cols]`) usando o `.predict` do modelo recém criado.

<!-- 
y_pred = model.predict(datasets["test"][feature_cols])
     -->

In [ ]:
y_true = datasets["test"]["target"].values
y_pred = ###

* cálculo do MAE e RMSE para nosso modelo no dataset de teste

In [ ]:
mean_absolute_error(y_true, y_pred)

In [ ]:
rmse(y_true, y_pred)

#### Quão bom é esse resultado?

* cálculo do $R^2$

* cálculo do RMSE para usando como valor predito o valor médio

In [ ]:
r2_score(y_true, y_pred)

**Valor médio do dataset de treino**

Esse seria uma solução _naive_ (inocente) que sempre devolve a mesma predição de prazo, não importa quais sejam as informações do `customer` e `seller`.

In [ ]:
train_dataset_target_mean = datasets["train"]["target"].mean()
mean_pred = [train_dataset_target_mean] * len(y_true)

In [ ]:
mean_absolute_error(y_true, mean_pred)

In [ ]:
rmse(y_true, mean_pred)

In [ ]:
def create_metrics_comparison_table(model, model_name, test_df, target_mean):
    metrics_results = {"model": [], "mae": [], "rmse": [], "r2": []}
    y_true = test_df["target"].values
    predictions = {
        model_name: model.predict(test_df[feature_cols]),
        "naive": [target_mean] * len(y_true)}
    for name, y_pred in predictions.items():
        metrics_results["model"].append(name)
        metrics_results["mae"].append(mean_absolute_error(y_true, y_pred))
        metrics_results["rmse"].append(rmse(y_true, y_pred))
        metrics_results["r2"].append(r2_score(y_true, y_pred))
    
    return pd.DataFrame(metrics_results)    

In [ ]:
create_metrics_comparison_table(model, "linear_regression", datasets["test"], train_dataset_target_mean)

### Resultados por estado de origem e destino

In [ ]:
frequent_from_to = datasets["test"]["from_to"].value_counts().head(n=5).index.tolist()

In [ ]:
for from_to_states in frequent_from_to:
    print(from_to_states)
    aux_df = datasets["test"][datasets["test"]["from_to"] == from_to_states]
    display(create_metrics_comparison_table(model, "linear_regression", aux_df))